In [1]:
from utils import read_jsonl, save_jsonl
import pandas as pd
from pydantic import BaseModel, model_validator, field_validator, Field, ValidationInfo, Extra
from typing import List, Dict, Union, Any, Optional, Literal
import instructor
from openai import OpenAI
import os
import json
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents.base import Document
from langchain_openai import OpenAIEmbeddings

from wikidata_search import WikidataSearch, get_all_properties_with_labels
from data_classes import KnowledgeGraph, ValidatedProperty
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.tools import DuckDuckGoSearchRun
import utils

In [2]:
save_path = '../../data/web_context_FB13_evaluation_results.jsonl'

In [29]:
client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))
MODEL = "gpt-3.5-turbo-0125"

# 🛜 Web Search 

In [4]:
# https://github.com/deedy5/duckduckgo_search?tab=readme-ov-file#TOP
from duckduckgo_search import DDGS

results = DDGS().text("Umberto I King of Italy cause of death?", max_results=5)
for r in results: print(r)

{'title': 'Umberto I of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Umberto_I_of_Italy', 'body': "Umberto I (Italian: Umberto Rainerio Carlo Emanuele Giovanni Maria Ferdinando Eugenio di Savoia; 14 March 1844 - 29 July 1900) was King of Italy from 9 January 1878 until his assassination in 1900. His reign saw Italy's expansion into the Horn of Africa, as well as the creation of the Triple Alliance among Italy, Germany and Austria-Hungary.. The son of Victor Emmanuel II and Adelaide of ..."}
{'title': 'Italian American assassinates Italian king | July 29, 1900 | HISTORY', 'href': 'https://www.history.com/this-day-in-history/italian-american-assassinates-italian-king', 'body': 'In Monza, Italy, King Umberto I is shot to death by Gaetano Bresci, an Italian-born anarchist who resided in America before returning to his homeland to murder the king. Crowned in 1878, King ...'}
{'title': 'This Is The Assassination Of King Umberto Of Italy Explained', 'href': 'https://www.grunge.c

# 🧠 Load data

In [30]:
neg_triples = utils.read_nell_sports('./NELL/neg.txt')
pos_triples = utils.read_nell_sports('./NELL/pos.txt')
# neg_triples[:15]

In [31]:
fb13_path = '../../data/FB13/test.tsv'
positive_triples, negative_triples = utils.read_fb13(fb13_path)

print("Positive Triples:")
for triple in positive_triples[:6]:
    print(triple)

print("\nNegative Triples:")
for triple in negative_triples[:6]:
    print(triple)

print(len(positive_triples) + len(negative_triples))

Positive Triples:
{'subject': 'umberto_i_of_italy', 'relation': 'cause_of_death', 'object': 'tyrannicide'}
{'subject': 'john_glenn_beall_jr', 'relation': 'nationality', 'object': 'united_states'}
{'subject': 'john_atkinson_grimshaw', 'relation': 'gender', 'object': 'male'}
{'subject': 'hardinge_giffard_1st_earl_of_halsbury', 'relation': 'gender', 'object': 'male'}
{'subject': 'mike_von_erich', 'relation': 'nationality', 'object': 'united_states'}
{'subject': 'gilbert_carlton_walker', 'relation': 'nationality', 'object': 'united_states'}

Negative Triples:
{'subject': 'umberto_i_of_italy', 'relation': 'cause_of_death', 'object': 'cerebral_aneurysm'}
{'subject': 'john_glenn_beall_jr', 'relation': 'nationality', 'object': 'ancient_greece'}
{'subject': 'john_atkinson_grimshaw', 'relation': 'gender', 'object': 'female'}
{'subject': 'hardinge_giffard_1st_earl_of_halsbury', 'relation': 'gender', 'object': 'female'}
{'subject': 'mike_von_erich', 'relation': 'nationality', 'object': 'serbia'}
{

# 🪬 Define Evaluation Model

In [35]:

class ValidatedProperty(BaseModel, extra=Extra.allow):
    entity_label: str
    property_name: str
    property_value: Any

    property_is_valid: Literal[True, False, "Not enough information to say"] = Field(
      ...,
        description="Whether the property is generally valid, judged against " +
                    "the given context.",
    )
    is_valid_reason: Optional[str] = Field(
        None, description="The reason why the property is valid if it is indeed valid."
    )
    error_message: Optional[str] = Field(
        None, description="The error message if either property_name and/or property_value is not valid."
    )


class WebKGValidator(BaseModel):

    triples: List
    validated_properties: List[ValidatedProperty] = []


    @staticmethod
    def get_web_search_results(search_tool, query):
        hit = search_tool.run(query)
        return hit

    @staticmethod
    def create_parent_document_retriever(docs: List[Document]):
        # https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever

        # This text splitter is used to create the child documents
        child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
        # The vectorstore to use to index the child chunks
        vectorstore = Chroma(
            collection_name="full_documents", embedding_function=OpenAIEmbeddings()
        )
        # The storage layer for the parent documents
        store = InMemoryStore()
        retriever = ParentDocumentRetriever(
            vectorstore=vectorstore,
            docstore=store,
            child_splitter=child_splitter,
            # parent_splitter=parent_splitter,
        )
        retriever.add_documents(docs, ids=None) # add entity doc(s)

        # list(store.yield_keys())   # see how many chunks it's created

        return retriever, store, vectorstore

    @staticmethod
    def retrieve_relevant_property(entity_name, property_name, vectorstore, retriever):
        '''Fetch the most similar chunk to predicted property name'''

        query = f"{property_name}"

        sub_docs = vectorstore.similarity_search(query)

        relevant_property = sub_docs[0].page_content
        return relevant_property

    @staticmethod
    def validate_statement_with_context(entity_label, predicted_property_name, predicted_property_value, context):
        '''Validate a statement about an entity

        a statement is a triple: entity_label --> predicted_property_name --> predicted_property_value
                             e.g Donald Trump --> wife --> Ivanka Trump
        
        '''
        resp: ValidatedProperty = client.chat.completions.create(
                        response_model=ValidatedProperty,
                        messages=[
                            {
                                "role": "user",
                                "content": f"Using the given context as a reference, " +
                                "is the following predicted property valid for the given entity? " +
                                f"\nEntity Label: {entity_label}" +
                                f"\nPredicted Property Name: {predicted_property_name}" +
                                f"\nPredicted Property Value: {predicted_property_value}" +
                                f"\n\nContext {context}"
                            }
                        ],
                        max_retries=2,
                        temperature=0,
                        model=MODEL,
                    )
        return resp


    @staticmethod
    def create_query(subject, relation, object):
        '''Create a query for the web search engine'''
        subject = " ".join(word.capitalize() for word in subject.split("_"))
        relation = " ".join(relation.split("_"))
        search_query = f"What {subject} {relation}?"
        return search_query

    @model_validator(mode='before')
    def validate(self, context) -> "WebKGValidator":

        self['validated_properties'] = []

        search_tool = DuckDuckGoSearchRun()

        for triple in self['triples']:

            subject, relation, object = triple['subject'], triple['relation'], triple['object']

            search_query = WebKGValidator.create_query(subject, relation, object)

            web_reference = WebKGValidator.get_web_search_results(search_tool, search_query)

            # EVALUATE ONE PROPERTY
            resp = WebKGValidator.validate_statement_with_context(
                entity_label=subject, 
                predicted_property_name=relation, 
                predicted_property_value=object, 
                context=web_reference
            )
            resp.sources = [web_reference]

            self['validated_properties'].append(resp)
        return self


    @model_validator(mode='after')
    def assert_all_properties_validated(self, info: ValidationInfo):
        if len(self.validated_properties) != len(self.triples):
            raise ValueError(
                "Number of properties validated does not match number of properties in the prediction knowledge base. " +
                f"Number of properties validated: {len(self.validated_properties)}, " +
                f"Number of properties in the text: {len(self.triples)}"
                )
        return self



/tmp/ipykernel_745231/1231154746.py:1: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  class ValidatedProperty(BaseModel, extra=Extra.allow):


In [36]:
idx = 7
subject, relation, object = neg_triples[idx]['subject'], neg_triples[idx]['relation'], neg_triples[idx]['object']

search_query = WebKGValidator.create_query(subject, relation, object)

search_tool = DuckDuckGoSearchRun()
web_reference = WebKGValidator.get_web_search_results(search_tool, search_query)

print(search_query)
print(web_reference)

What Arkansas Razorbacks ,?
Get the latest news and information for the Arkansas Razorbacks. 2023 season schedule, scores, stats, and highlights. Find out the latest on your favorite NCAAF teams on CBSSports.com. Get the latest news and information for the Arkansas Razorbacks. 2023 season schedule, scores, stats, and highlights. Find out the latest on your favorite NCAAB teams on CBSSports.com. The Razorbacks are unranked with a .500 record, standing 14-14. Their poor 5-10 SEC record places them in third-to-last place in the conference, overcoming only Missouri and Vanderbilt. But when the Razorbacks lost to lowly Vanderbilt on Tuesday at home, Kentucky's chances of beating Arkansas grew even larger. Now, according to ESPN's Basketball Power Index, the Wildcats have a 93% chance of winning Saturday's matinee. Arkansas had been playing good basketball until the Commodores arrived in Fayetteville. Full Arkansas Razorbacks schedule for the 2023-24 season including dates, opponents, game t

# Evaluate!

In [37]:
neg_results = []

neg_results.append(WebKGValidator(**{'triples': negative_triples[:7]}))

neg_results[0].model_dump()['validated_properties']

[{'entity_label': 'umberto_i_of_italy',
  'property_name': 'cause_of_death',
  'property_value': 'cerebral_aneurysm',
  'property_is_valid': False,
  'is_valid_reason': None,
  'error_message': 'The predicted cause of death for Umberto I of Italy is incorrect. Umberto I was assassinated by an anarchist, Gaetano Bresci, and not due to a cerebral aneurysm.',
  'sources': ["The prince accepted civil liability over the death and was charged and indicted, but he was ultimately acquitted of fatal wounding and unintentional homicide by a court in Paris 13 years later. February 21, 2024. Photo: Getty Images. This story begins on July 29, 1900, in the city of Monza in Italy's Lombardy region, when King Umberto I was assassinated by the anarchist Gaetano Bresci ... ROME (AP) — Prince Vittorio Emanuele of Savoy, the son of Italy's last king, Umberto II, has died at the age of 86. The prince died on Saturday in Geneva, the Savoy Royal House said in a statement. He had lived in Switzerland since th

In [38]:
pos_results = []

pos_results.append(WebKGValidator(**{'triples': positive_triples[:7]}))

pos_results[0].model_dump()['validated_properties']

[{'entity_label': 'umberto_i_of_italy',
  'property_name': 'cause_of_death',
  'property_value': 'tyrannicide',
  'property_is_valid': False,
  'is_valid_reason': None,
  'error_message': "The predicted property 'cause_of_death' with the value 'tyrannicide' is not valid for the entity 'umberto_i_of_italy'. Umberto I of Italy was assassinated by an anarchist, Gaetano Bresci, and not by an act of tyrannicide.",
  'sources': ["He was the only son among the four children born to Umberto II and his wife, Belgian Princess Marie-Jose. The prince of Naples may have won the birth lottery but within a few short years, a ... February 21, 2024. Photo: Getty Images. This story begins on July 29, 1900, in the city of Monza in Italy's Lombardy region, when King Umberto I was assassinated by the anarchist Gaetano Bresci ... ROME (AP) — Prince Vittorio Emanuele of Savoy, the son of Italy's last king, Umberto II, has died at the age of 86. The prince died on Saturday in Geneva, the Savoy Royal House sai

In [43]:
tp = 0
fp = 0
tn = 0
fn = 0 
for val in pos_results[0].model_dump()['validated_properties']:
    if val['property_is_valid']:    # property is correctly marked as valid
        tp += 1
    else:                           # property is incorrectly marked as invalid
        fn += 1
for val in neg_results[0].model_dump()['validated_properties']:
    if val['property_is_valid']:    # property is incorrectly marked as valid
        fp += 1
    else:                           # property is correctly marked as invalid
        tn += 1

metrics = utils.calc_metrics(tp, fp, tn, fn)

Precision: 0.75
Recall: 0.8571428571428571
F1 Score: 0.7999999999999999
----------


In [44]:
results_json = [r.model_dump() for r in neg_results] + [r.model_dump() for r in pos_results]
save_jsonl(results_json, save_path)

Saved to f'../../data/web_context_FB13_evaluation_results.jsonl


# Look at our Evaluations

In [3]:
results = read_jsonl(save_path)
len(results)

2

In [4]:
results[0]

{'triples': [{'subject': 'umberto_i_of_italy',
   'relation': 'cause_of_death',
   'object': 'cerebral_aneurysm'},
  {'subject': 'john_glenn_beall_jr',
   'relation': 'nationality',
   'object': 'ancient_greece'},
  {'subject': 'john_atkinson_grimshaw',
   'relation': 'gender',
   'object': 'female'},
  {'subject': 'hardinge_giffard_1st_earl_of_halsbury',
   'relation': 'gender',
   'object': 'female'},
  {'subject': 'mike_von_erich', 'relation': 'nationality', 'object': 'serbia'},
  {'subject': 'gilbert_carlton_walker',
   'relation': 'nationality',
   'object': 'barbados'},
  {'subject': 'jorge_gonzalez_camarena',
   'relation': 'gender',
   'object': 'female'}],
 'validated_properties': [{'entity_label': 'umberto_i_of_italy',
   'property_name': 'cause_of_death',
   'property_value': 'cerebral_aneurysm',
   'property_is_valid': False,
   'is_valid_reason': None,
   'error_message': 'The predicted cause of death for Umberto I of Italy is incorrect. Umberto I was assassinated by an an

In [5]:
tp = 0
fp = 0
tn = 0
fn = 0 
pos_results = results[1]
neg_results = results[0]
for val in pos_results['validated_properties']:
    if val['property_is_valid']:    # property is correctly marked as valid
        tp += 1
    else:                           # property is incorrectly marked as invalid
        fn += 1
for val in neg_results['validated_properties']:
    if val['property_is_valid']:    # property is incorrectly marked as valid
        fp += 1
    else:                           # property is correctly marked as invalid
        tn += 1

metrics = utils.calc_metrics(tp, fp, tn, fn)

# Precision: 0.75
# Recall: 0.8571428571428571
# F1 Score: 0.7999999999999999
# Accuracy: 0.7857142857142857
# ----------

Precision: 0.75
Recall: 0.8571428571428571
F1 Score: 0.7999999999999999
Accuracy: 0.7857142857142857
----------


{'precision': 0.75,
 'recall': 0.8571428571428571,
 'f1_score': 0.7999999999999999,
 'accuracy': 0.7857142857142857}